# 1. 기본적인 DNN

# 2. 사이킷런(sklearn) 이용
- One-Hot Encoding 하지 않고 Label Encoding까지만 해야 작동. One-Hot Encoding은 작동되지 않음

In [1]:
from sklearn import datasets # iris 데이터 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [2]:
# 1. Dataset
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print(iris_X[::50], iris_y[::50])
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, 
                                                    test_size = .1, # 기본값은 train:test = 0.75:0.25
                                                    random_state = 4,
                                                    stratify = iris_y)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

[[5.1 3.5 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.3 3.3 6.  2.5]] [0 1 2]
(135, 4) (135,) (15, 4) (15,)


In [69]:
# 사이킷런에서 제공하는 다층퍼셉트론 기반의 분류 알고리즘 API
# 생성자함수
# mlp 변수가 model과 같은 기능
mlp = MLPClassifier(hidden_layer_sizes = (50, 30),
                    activation = 'relu',
                    solver = 'adam',           # optimizer 설정값
                    max_iter = 1000,           # 최대학습횟수
                    early_stopping = True,     # 조기 종료 활성화
                    n_iter_no_change = 150,     # 검증셋 손실함수가 개선되지 않으면 조기종료 (=patience)
                    validation_fraction = 0.1) # 검증셋 비율

In [70]:
# validation data를 넣을 수 없음
# seed값 설정 가능
mlp.fit(train_X, train_y)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(50, 30), max_iter=1000,
              n_iter_no_change=150)

In [71]:
mlp.score(test_X, test_y)

0.9333333333333333

In [51]:
test_X[0], test_X[0:1], test_X, test_X # 슬라이싱하면 1차원 → 2차원

(array([5.1, 3.8, 1.6, 0.2]),
 array([[5.1, 3.8, 1.6, 0.2]]),
 array([[5.1, 3.8, 1.6, 0.2],
        [6.3, 3.3, 6. , 2.5],
        [6.3, 2.5, 4.9, 1.5],
        [5.7, 2.5, 5. , 2. ],
        [6.4, 3.2, 5.3, 2.3],
        [6.1, 2.8, 4.7, 1.2],
        [6.8, 3.2, 5.9, 2.3],
        [7.7, 2.8, 6.7, 2. ],
        [5.1, 3.3, 1.7, 0.5],
        [5.4, 3.9, 1.7, 0.4],
        [4.9, 3. , 1.4, 0.2],
        [5. , 3.5, 1.3, 0.3],
        [5.5, 2.3, 4. , 1.3],
        [6.3, 3.3, 4.7, 1.6],
        [5.9, 3. , 4.2, 1.5]]),
 array([[5.1, 3.8, 1.6, 0.2],
        [6.3, 3.3, 6. , 2.5],
        [6.3, 2.5, 4.9, 1.5],
        [5.7, 2.5, 5. , 2. ],
        [6.4, 3.2, 5.3, 2.3],
        [6.1, 2.8, 4.7, 1.2],
        [6.8, 3.2, 5.9, 2.3],
        [7.7, 2.8, 6.7, 2. ],
        [5.1, 3.3, 1.7, 0.5],
        [5.4, 3.9, 1.7, 0.4],
        [4.9, 3. , 1.4, 0.2],
        [5. , 3.5, 1.3, 0.3],
        [5.5, 2.3, 4. , 1.3],
        [6.3, 3.3, 4.7, 1.6],
        [5.9, 3. , 4.2, 1.5]]))

In [52]:
mlp.predict(test_X[0:1]) # 2차원 데이터

array([0])

# 3. 클래스를 생성

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [29]:
class DNNClassifier:
    @staticmethod
    def build(input_dim = 4, activation = 'relu', output_dim = 3):
#         model = Sequential()
#         model.add(Input(input_dim, ))
#         model.add(Dense(50, activation = activation))
#         model.add(Dense(30, activation = activation))
#         model.add(Dense(output_dim, activation = 'softmax'))
        model = Sequential([
            Input(input_dim, ),
            Dense(50, activation = activation),
            Dropout(0.2),
            Dense(30, activation = activation),
            Dense(output_dim, activation = 'softmax')
        ])
        # 학습 과정 설정
        model.compile(loss = 'categorical_crossentropy',
                      optimizer = 'adam',
                      metrics = ['accuracy'])
        return model

In [75]:
# 1. 데이터 생성 및 전처리
# iris_Y = pd.get_dummies(iris_y).to_numpy()
# train_X, test_X, train_Y, test_Y = train_test_split(iris_X, iris_Y, # One-Hot Encoded Data로 분류
#                                                     test_size = .1, # 기본값은 train:test = 0.75:0.25
#                                                     random_state = 4,
#                                                     stratify = iris_y)
# print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)
train_Y = pd.get_dummies(train_y).values
test_Y  = pd.get_dummies(test_y).values

# 2. 모델 생성
model = DNNClassifier.build() # 기본값 (input_dim = 4, activation = 'relu', output_dim = 3)
# 3. 모델 학습
earlyStopping = EarlyStopping(patience = 150) # 객체 생성
hist = model.fit(train_X, train_Y, 
                 epochs = 1000, 
                 validation_split = .1, 
                 callbacks = [earlyStopping])

Epoch 1/1000
4/4 [==============================] - 1s 61ms/step - loss: 1.0330 - accuracy: 0.3554 - val_loss: 0.9549 - val_accuracy: 0.5714
Epoch 2/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.9080 - accuracy: 0.5868 - val_loss: 0.9263 - val_accuracy: 0.7857
Epoch 3/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.8618 - accuracy: 0.6529 - val_loss: 0.8768 - val_accuracy: 0.7857
Epoch 4/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.8257 - accuracy: 0.6529 - val_loss: 0.7972 - val_accuracy: 0.7857
Epoch 5/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.7813 - accuracy: 0.6529 - val_loss: 0.7210 - val_accuracy: 0.7857
Epoch 6/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.7463 - accuracy: 0.6529 - val_loss: 0.6557 - val_accuracy: 0.7857
Epoch 7/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.7116 - accuracy: 0.6777 - val_loss: 0.6035 - val_accuracy: 0.7857
Epoch 8/1000


Epoch 59/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1814 - accuracy: 0.9835 - val_loss: 0.1509 - val_accuracy: 1.0000
Epoch 60/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.1805 - accuracy: 0.9835 - val_loss: 0.1590 - val_accuracy: 0.9286
Epoch 61/1000
4/4 [==============================] - 0s 9ms/step - loss: 0.1755 - accuracy: 0.9835 - val_loss: 0.1470 - val_accuracy: 1.0000
Epoch 62/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.1686 - accuracy: 0.9835 - val_loss: 0.1424 - val_accuracy: 1.0000
Epoch 63/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1672 - accuracy: 0.9835 - val_loss: 0.1387 - val_accuracy: 1.0000
Epoch 64/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.1622 - accuracy: 0.9835 - val_loss: 0.1400 - val_accuracy: 1.0000
Epoch 65/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.1606 - accuracy: 0.9835 - val_loss: 0.1478 - val_accuracy: 0.9286
Epoch 6

4/4 [==============================] - 0s 11ms/step - loss: 0.0640 - accuracy: 0.9835 - val_loss: 0.1010 - val_accuracy: 0.9286
Epoch 174/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.0625 - accuracy: 0.9917 - val_loss: 0.0998 - val_accuracy: 1.0000
Epoch 175/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0598 - accuracy: 0.9917 - val_loss: 0.1011 - val_accuracy: 0.9286
Epoch 176/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.0635 - accuracy: 0.9835 - val_loss: 0.1099 - val_accuracy: 0.9286
Epoch 177/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.0621 - accuracy: 0.9835 - val_loss: 0.1013 - val_accuracy: 0.9286
Epoch 178/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.0593 - accuracy: 0.9917 - val_loss: 0.1008 - val_accuracy: 0.9286
Epoch 179/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.0594 - accuracy: 0.9917 - val_loss: 0.1007 - val_accuracy: 1.0000
Epoch 180/1000

In [76]:
loss, accuracy = model.evaluate(test_X, test_Y)
print(f'정확도: {accuracy*100}%')

1/1 [==============================] - 0s 36ms/step - loss: 0.0491 - accuracy: 1.0000
정확도: 100.0%


In [78]:
model.predict(test_X[0].reshape(1, -1)).argmax(axis = 1)

1/1 [==============================] - 0s 27ms/step


array([0], dtype=int64)

# 4. 함수 API 이용: 병렬처리

In [81]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model # 생성자함수

In [79]:
train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

((135, 4), (135, 3), (15, 4), (15, 3))

In [85]:
# 모델
# Dense층을 병렬로 계산
# parameter 수 계산
input_ = Input(shape = (4, ))
dense1 = Dense(50, activation = 'relu')(input_) # 4x50 + 50 = 250
dense2 = Dense(80, activation = 'relu')(input_) # 4x80 + 80 = 400
dense3 = Dense(30, activation = 'relu')(input_) # 4x30 + 30 = 150

# Dense 연결(Input 4로 만든 3개의 w, b 합치기)
# 이미지 학습 시 Dense층 병렬로 만들면 속도 빨라짐(결과는 동일)
# x = Dense(160, activation = 'relu')(input_)의 병렬처리 구현
x = concatenate([dense1, dense2, dense3])  
output = Dense(units = 3, activation = 'softmax')(x) # 160x3 + 3 = 483
model = Model(inputs  = input_, 
              outputs = output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_72 (Dense)               (None, 50)           250         ['input_25[0][0]']               
                                                                                                  
 dense_73 (Dense)               (None, 80)           400         ['input_25[0][0]']               
                                                                                                  
 dense_74 (Dense)               (None, 30)           150         ['input_25[0][0]']               
                                                                                            

In [86]:
# 학습 정의(설정), 학습
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
model.fit(train_X, train_Y, 
          epochs = 200,
          verbose = 1)

Epoch 1/200
5/5 [==============================] - 0s 5ms/step - loss: 1.0564 - accuracy: 0.3333
Epoch 2/200
5/5 [==============================] - 0s 8ms/step - loss: 0.8853 - accuracy: 0.7556
Epoch 3/200
5/5 [==============================] - 0s 8ms/step - loss: 0.8163 - accuracy: 0.8963
Epoch 4/200
5/5 [==============================] - 0s 5ms/step - loss: 0.7378 - accuracy: 0.9704
Epoch 5/200
5/5 [==============================] - 0s 5ms/step - loss: 0.6854 - accuracy: 0.7037
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 0.6446 - accuracy: 0.7037
Epoch 7/200
5/5 [==============================] - 0s 8ms/step - loss: 0.5970 - accuracy: 0.9556
Epoch 8/200
5/5 [==============================] - 0s 9ms/step - loss: 0.5683 - accuracy: 0.8815
Epoch 9/200
5/5 [==============================] - 0s 6ms/step - loss: 0.5337 - accuracy: 0.9333
Epoch 10/200
5/5 [==============================] - 0s 5ms/step - loss: 0.5185 - accuracy: 0.9333
Epoch 11/200
5/5 [===========

5/5 [==============================] - 0s 5ms/step - loss: 0.0862 - accuracy: 0.9852
Epoch 168/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0928 - accuracy: 0.9704
Epoch 169/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0899 - accuracy: 0.9852
Epoch 170/200
5/5 [==============================] - 0s 5ms/step - loss: 0.0895 - accuracy: 0.9778
Epoch 171/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0879 - accuracy: 0.9778
Epoch 172/200
5/5 [==============================] - 0s 7ms/step - loss: 0.0870 - accuracy: 0.9852
Epoch 173/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0876 - accuracy: 0.9852
Epoch 174/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0874 - accuracy: 0.9852
Epoch 175/200
5/5 [==============================] - 0s 8ms/step - loss: 0.0856 - accuracy: 0.9778
Epoch 176/200
5/5 [==============================] - 0s 5ms/step - loss: 0.0877 - accuracy: 0.9852
Epoch 177/200
5/5 [=====

In [87]:
# 학습평가
loss, accuracy = model.evaluate(test_X, test_Y)
print(accuracy*100, '%')

1/1 [==============================] - 0s 143ms/step - loss: 0.0688 - accuracy: 0.9333
93.33333373069763 %
